In [1]:
%load_ext Cython

In [6]:
%%cython --annotate

#from __future__ import division

cimport cython

from libc.math cimport ceil, sqrt

import numpy as np
cimport numpy as np

ITYPE = np.int64
ctypedef np.int_t ITYPE_t
FTYPE = np.float64
ctypedef np.float64_t FTYPE_t


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def sphbin2cartbin(FTYPE_t r_max, ITYPE_t r_power,
                   ITYPE_t n_rbins, ITYPE_t n_thetabins,
                   FTYPE_t x_bw, FTYPE_t y_bw, FTYPE_t z_bw,
                   ITYPE_t x_oversample, ITYPE_t y_oversample,
                   ITYPE_t z_oversample,
                   ITYPE_t antialias_factor=1):
    """
    Parameters
    ----------
    r_edges_mg, theta_edges_mg, phi_edges_mg : numpy.ndarray
        Meshgrid of r, theta, and phi edges

    x_bw, y_bw, z_bw : float
        Cartesian binwidths in x, y, and z directions

    x_oversample, y_oversample, z_oversample : int
        Oversmapling factors. If oversampling is used, the returned indices
        array will have floating point values. E.g., a bin index with
        oversampling of 2 could have take values 0, 0.5, 1, ...
        Note that this increases the computational cost _and_ increases the
        memory footprint of the produced array(s).

    antialias_factor : int
        The smallest binning unit in each dimension is divided again by
        this factor for more accruately computing the volume of overlap
        (and then the sub-binning for antialiasing is discarded). This
        therefore does not add to the memory footprint, but will increase
        the computational cost.

    Returns
    -------
    indices : list of M shape (N x 3) numpy.ndarrays
        One array per spherical bin. Data type of the arrays is int32 if
        all oversample factors are set to 1; otherwise, dtype is float64.

    overlap_vol : list of M shape (N,) numpy.ndarrays, dtype of float64
        One array per spherical bin

    """
    cdef FTYPE_t x_bw_os = x_bw / FTYPE(x_oversample)
    cdef FTYPE_t y_bw_os = y_bw / FTYPE(y_oversample)
    cdef FTYPE_t z_bw_os = z_bw / FTYPE(z_oversample)

    cdef FTYPE_t x_bw_os_aa = x_bw_os / FTYPE(antialias_factor)
    cdef FTYPE_t y_bw_os_aa = y_bw_os / FTYPE(antialias_factor)
    cdef FTYPE_t z_bw_os_aa = z_bw_os / FTYPE(antialias_factor)

    cdef FTYPE_t aa_vol = x_bw_os_aa * y_bw_os_aa * z_bw_os_aa
    
    cdef FTYPE_t x_halfbw_os_aa = x_bw_os_aa / 2.0
    cdef FTYPE_t y_halfbw_os_aa = y_bw_os_aa / 2.0
    cdef FTYPE_t z_halfbw_os_aa = z_bw_os_aa / 2.0

    cdef ITYPE_t n_xbins_oct_os = int(ceil(r_max / x_bw_os))
    cdef ITYPE_t n_ybins_oct_os = int(ceil(r_max / y_bw_os))
    cdef ITYPE_t n_zbins_oct_os = int(ceil(r_max / z_bw_os))

    cdef ITYPE_t n_xbins_oct_os_aa = int(ceil(r_max / x_bw_os_aa))
    cdef ITYPE_t n_ybins_oct_os_aa = int(ceil(r_max / y_bw_os_aa))
    cdef ITYPE_t n_zbins_oct_os_aa = int(ceil(r_max / z_bw_os_aa))

    cdef FTYPE_t r_max_sq = r_max**2
    cdef FTYPE_t sqrt_r_bin_scale = FTYPE(n_rbins) / sqrt(r_max)
    cdef FTYPE_t costheta_bin_scale = FTYPE(n_thetabins) / 2.0

    # Variables used below must be declared here to have type declarations
    
    cdef ITYPE_t x_os_idx
    cdef ITYPE_t y_os_idx
    cdef ITYPE_t z_os_idx
    
    cdef FTYPE_t x_idx
    cdef FTYPE_t y_idx
    cdef FTYPE_t z_idx
    
    cdef ITYPE_t xi
    cdef ITYPE_t yi
    cdef ITYPE_t zi

    cdef ITYPE_t r_bin_idx
    cdef ITYPE_t theta_bin_idx
    cdef ITYPE_t flat_bin_idx
    
    cdef FTYPE_t x0
    cdef FTYPE_t y0
    cdef FTYPE_t z0
    
    cdef np.ndarray x_centers = np.empty(antialias_factor, dtype=FTYPE)
    cdef np.ndarray x_centers_sq = np.empty(antialias_factor, dtype=FTYPE)
    cdef np.ndarray rho_squares = np.empty((antialias_factor, antialias_factor), dtype=FTYPE)
    
    cdef FTYPE_t x_center
    cdef FTYPE_t x_center_sq
    cdef FTYPE_t y_center
    cdef FTYPE_t y_center_sq
    cdef FTYPE_t z_center
    cdef FTYPE_t z_center_sq

    cdef FTYPE_t r
    cdef FTYPE_t old_vol
    cdef FTYPE_t new_vol
    
    # Note: there is no phi dependence in the tables
    
    bin_mapping = []
    for _ in range(n_rbins):
        tmp = []
        for _ in range(n_thetabins):
            tmp.append({})
        bin_mapping.append(tmp)
        
    for x_os_idx in range(n_xbins_oct_os):
        x_idx = FTYPE(x_os_idx) / FTYPE(x_oversample)
        x0 = x_os_idx * x_bw_os + x_halfbw_os_aa

        for xi in range(antialias_factor):
            x_center = x0 + xi * x_bw_os_aa
            x_center_sq = x_center * x_center
            x_centers[xi] = x_center
            x_centers_sq[xi] = x_center_sq

        for y_os_idx in range(n_ybins_oct_os):
            y_idx = FTYPE(y_os_idx) / FTYPE(y_oversample)

            y0 = y_os_idx * y_bw_os + y_halfbw_os_aa
            for yi in range(antialias_factor):
                y_center = y0 + yi * y_bw_os_aa
                y_center_sq = y_center * y_center
                for xi in range(antialias_factor):
                    rho_squares[xi, yi] = x_centers_sq[xi] + y_center_sq

            for z_os_idx in range(n_zbins_oct_os):
                z_idx = FTYPE(z_os_idx) / FTYPE(z_oversample)
                
                xyz_idx_q1 = (x_idx, y_idx, z_idx)
                xyz_idx_q2 = (-x_idx, y_idx, z_idx)
                xyz_idx_q3 = (-x_idx, -y_idx, z_idx)
                xyz_idx_q4 = (x_idx, -y_idx, z_idx)
                
                xyz_idx_negz_q1 = (x_idx, y_idx, -z_idx)
                xyz_idx_negz_q2 = (-x_idx, y_idx, -z_idx)
                xyz_idx_negz_q3 = (-x_idx, -y_idx, -z_idx)
                xyz_idx_negz_q4 = (x_idx, -y_idx, -z_idx)

                z0 = z_os_idx * z_bw_os + z_halfbw_os_aa
                for zi in range(antialias_factor):
                    z_center = z0 + zi * z_bw_os_aa
                    z_center_sq = z_center * z_center
                    for xi in range(antialias_factor):
                        for yi in range(antialias_factor):
                            r = sqrt(rho_squares[xi, yi] + z_center_sq)
                            if r >= r_max:
                                continue
                            r_bin_idx = int(sqrt(r) * sqrt_r_bin_scale)
                            theta_bin_idx = int((1.0 - z_center / r) * costheta_bin_scale)
                            if theta_bin_idx < 0 or theta_bin_idx >= n_thetabins:
                                continue
                           
                            d = bin_mapping[r_bin_idx][theta_bin_idx]
                            new_vol = aa_vol + d.get(xyz_idx_q1, FTYPE(0.0))
                            d[xyz_idx_q1] = new_vol
                            d[xyz_idx_q2] = new_vol
                            d[xyz_idx_q3] = new_vol
                            d[xyz_idx_q4] = new_vol

                            theta_bin_idx = n_thetabins - theta_bin_idx - 1
                            d[xyz_idx_negz_q1] = new_vol
                            d[xyz_idx_negz_q2] = new_vol
                            d[xyz_idx_negz_q3] = new_vol
                            d[xyz_idx_negz_q4] = new_vol 
                            
    return bin_mapping

In [7]:
import os
from os.path import abspath, dirname
os.sys.path.append(dirname(dirname(abspath('__file__'))))

from retro import powerspace

In [11]:
r_max = 9; r_power = 2
n_rbins=10
n_thetabins = 10

r_edges = powerspace(0, r_max, n_rbins + 1, r_power)
theta_edges = np.arccos(np.linspace(1, -1, n_thetabins + 1))

print 'r_edges:', r_edges
print 'theta_edges:', theta_edges

R, THETA = np.meshgrid(r_edges, theta_edges, indexing='ij')
for ri in range(n_rbins):
    for ti in range(n_thetabins):
        rs = R[ri:ri+2, ti:ti+2]
        ts = THETA[ri:ri+2, ti:ti+2]
        bin_corner_coords = zip(rs.flat, ts.flat)
        print 'sph bin idx [r=%d, theta=%d] bin extents: (%.2f m - %.2f m, %.2f rad - %.2f rad)' \
                % (ri, ti, bin_corner_coords[0][0], bin_corner_coords[-1][0],
                   bin_corner_coords[0][1], bin_corner_coords[-1][1])

r_edges: [ 0.    0.09  0.36  0.81  1.44  2.25  3.24  4.41  5.76  7.29  9.  ]
theta_edges: [ 0.          0.64350111  0.92729522  1.15927948  1.36943841  1.57079633
  1.77215425  1.98231317  2.21429744  2.49809154  3.14159265]
sph bin idx [r=0, theta=0] bin extents: (0.00 m - 0.09 m, 0.00 rad - 0.64 rad)
sph bin idx [r=0, theta=1] bin extents: (0.00 m - 0.09 m, 0.64 rad - 0.93 rad)
sph bin idx [r=0, theta=2] bin extents: (0.00 m - 0.09 m, 0.93 rad - 1.16 rad)
sph bin idx [r=0, theta=3] bin extents: (0.00 m - 0.09 m, 1.16 rad - 1.37 rad)
sph bin idx [r=0, theta=4] bin extents: (0.00 m - 0.09 m, 1.37 rad - 1.57 rad)
sph bin idx [r=0, theta=5] bin extents: (0.00 m - 0.09 m, 1.57 rad - 1.77 rad)
sph bin idx [r=0, theta=6] bin extents: (0.00 m - 0.09 m, 1.77 rad - 1.98 rad)
sph bin idx [r=0, theta=7] bin extents: (0.00 m - 0.09 m, 1.98 rad - 2.21 rad)
sph bin idx [r=0, theta=8] bin extents: (0.00 m - 0.09 m, 2.21 rad - 2.50 rad)
sph bin idx [r=0, theta=9] bin extents: (0.00 m - 0.09 m, 2.50 r

In [18]:
%%time

bin_mapping = sphbin2cartbin(
    r_max=r_max, r_power=r_power,
    n_rbins=n_rbins, n_thetabins=n_thetabins,
    x_bw=1.0, y_bw=1.0, z_bw=1.0,
    x_oversample=1, y_oversample=1, z_oversample=1,
    antialias_factor=10
)

CPU times: user 588 ms, sys: 0 ns, total: 588 ms
Wall time: 592 ms


In [19]:
bin_mapping

[[{},
  {},
  {(0.0, 0.0, 0.0): 0.0010000000000000002},
  {},
  {},
  {},
  {},
  {},
  {},
  {}],
 [{(0.0, 0.0, 0.0): 0.005000000000000001},
  {(0.0, 0.0, 0.0): 0.005000000000000001},
  {(0.0, 0.0, 0.0): 0.005000000000000001},
  {(0.0, 0.0, 0.0): 0.003000000000000001},
  {(0.0, 0.0, 0.0): 0.007000000000000001},
  {},
  {},
  {},
  {},
  {}],
 [{(0.0, 0.0, 0.0): 0.05300000000000004},
  {(0.0, 0.0, 0.0): 0.048000000000000036},
  {(0.0, 0.0, 0.0): 0.05300000000000004},
  {(0.0, 0.0, 0.0): 0.05200000000000004},
  {(0.0, 0.0, 0.0): 0.05200000000000004},
  {},
  {},
  {},
  {},
  {}],
 [{(0.0, 0.0, -1.0): 0.16700000000000012,
   (0.0, 0.0, 0.0): 0.08800000000000006,
   (0.0, 0.0, 1.0): 0.16700000000000012},
  {(-1.0, 0.0, 0.0): 0.007000000000000001,
   (-0.0, -1.0, 0.0): 0.007000000000000001,
   (0.0, 0.0, -1.0): 0.03100000000000002,
   (0.0, 0.0, 0.0): 0.21500000000000016,
   (0.0, 0.0, 1.0): 0.03100000000000002,
   (0.0, 1.0, 0.0): 0.007000000000000001,
   (1.0, 0.0, 0.0): 0.0070000000000

8000